In [1]:
import logging

In [2]:
logging.basicConfig(level=logging.DEBUG,
                    format='%(asctime)s - %(name)s - %(levelname)s - %(message)s')

In [3]:
from telegram.ext import Updater


In [4]:
updater = Updater(token="484949622:AAGVa89zUGszMMp9D4Y48k-eFcw3xyQPukQ")

2018-02-28 21:43:55,034 - telegram.ext.dispatcher - DEBUG - Setting singleton dispatcher as <telegram.ext.dispatcher.Dispatcher object at 0x7f8cc590d1d0>


In [5]:
def start(bot, update):
    bot.send_message(chat_id=update.message.chat_id, text="I'm a bot, please talk to me!")

In [6]:
from telegram.ext import CommandHandler

In [7]:
start_handler = CommandHandler('start', start)

In [8]:
dispatcher = updater.dispatcher
dispatcher.add_handler(start_handler)

In [9]:
def echo(bot, update):
    bot.send_message(chat_id=update.message.chat_id, text=update.message.text)

In [10]:
from telegram.ext import MessageHandler, Filters
echo_handler = MessageHandler(Filters.text, echo)
dispatcher.add_handler(echo_handler)

In [14]:
updater.start_polling()

2018-02-28 21:50:38,983 - JobQueue - DEBUG - JobQueue thread started
2018-02-28 21:50:38,986 - telegram.ext.updater - DEBUG - dispatcher - started
2018-02-28 21:50:38,987 - telegram.ext.updater - DEBUG - updater - started
2018-02-28 21:50:38,994 - telegram.ext.updater - DEBUG - Updater thread started
2018-02-28 21:50:38,995 - telegram.bot - DEBUG - Entering: set_webhook
2018-02-28 21:50:38,998 - telegram.vendor.ptb_urllib3.urllib3.connectionpool - DEBUG - Starting new HTTPS connection (1): api.telegram.org
2018-02-28 21:50:38,999 - telegram.ext.dispatcher - DEBUG - Dispatcher started


2018-02-28 21:50:39,250 - telegram.vendor.ptb_urllib3.urllib3.connectionpool - DEBUG - https://api.telegram.org:443 "POST /bot484949622:AAGVa89zUGszMMp9D4Y48k-eFcw3xyQPukQ/setWebhook HTTP/1.1" 200 68
2018-02-28 21:50:39,255 - telegram.bot - DEBUG - True
2018-02-28 21:50:39,259 - telegram.bot - DEBUG - Exiting: set_webhook
2018-02-28 21:50:39,264 - telegram.bot - DEBUG - Entering: get_updates
2018-02-28 21:50:39,323 - telegram.vendor.ptb_urllib3.urllib3.connectionpool - DEBUG - https://api.telegram.org:443 "POST /bot484949622:AAGVa89zUGszMMp9D4Y48k-eFcw3xyQPukQ/getUpdates HTTP/1.1" 200 516
2018-02-28 21:50:39,324 - telegram.bot - DEBUG - Getting updates: [976005232]
2018-02-28 21:50:39,325 - telegram.bot - DEBUG - [<telegram.update.Update object at 0x7f8cc58aa358>]
2018-02-28 21:50:39,326 - telegram.bot - DEBUG - Exiting: get_updates
2018-02-28 21:50:39,327 - telegram.bot - DEBUG - Entering: get_updates
2018-02-28 21:50:39,327 - telegram.ext.dispatcher - DEBUG - Processing Update: {'upd

In [15]:
updater.stop()

2018-02-28 21:52:27,510 - JobQueue - DEBUG - JobQueue thread stopped
2018-02-28 21:52:27,518 - telegram.ext.updater - DEBUG - Stopping Updater and Dispatcher...
2018-02-28 21:52:27,522 - telegram.ext.updater - DEBUG - Requesting Dispatcher to stop...
2018-02-28 21:52:27,704 - telegram.ext.dispatcher - DEBUG - orderly stopping
2018-02-28 21:52:27,708 - telegram.ext.dispatcher - DEBUG - Dispatcher thread stopped
2018-02-28 21:52:27,714 - telegram.ext.updater - DEBUG - dispatcher - ended
2018-02-28 21:52:27,725 - telegram.ext.dispatcher - DEBUG - Waiting for async thread 1/4 to end
2018-02-28 21:52:27,726 - telegram.ext.dispatcher - DEBUG - Closing run_async thread 79101706-9f3e-4136-b420-b8a6c7b0a527_1/4
2018-02-28 21:52:27,726 - telegram.ext.dispatcher - DEBUG - Closing run_async thread 79101706-9f3e-4136-b420-b8a6c7b0a527_2/4
2018-02-28 21:52:27,727 - telegram.ext.dispatcher - DEBUG - Closing run_async thread 79101706-9f3e-4136-b420-b8a6c7b0a527_3/4
2018-02-28 21:52:27,727 - telegram.e